In [60]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [109]:
def scrape_coches_ocasion(url, diccionario):

    llamada = requests.get(url)  # Realiza una solicitud GET a la URL proporcionada y almacena la respuesta en 'llamada'.

    if llamada.status_code != 200:  # Comprueba si la respuesta no fue exitosa (código de estado distinto de 200).
            print(f"El motivo REASON por el que la llamada falló es {llamada.reason}")  # Imprime la razón de la falla.
            print(f"El motivo TEXT por el que la llamada falló es {llamada.text}")  # Imprime la razón de la falla.
            return None
    else:
        soup = BeautifulSoup(llamada.text, 'html.parser')
        spans = soup.find_all('span', {"class": "cardVehicle_spot__e6YZx"}) # Sopa de coches y precios originales
        precios_descuento = soup.find_all('span', {"class":"cardVehicle_finance__SG6JV"}) # Sopa de precios con descuento
        km = soup.find_all('span', {"class":"characteristics_elements__Mb1S_"}) # Sopa de km

        for kilom in km: # Metiendo los Km en el diccionario
            if kilom.get_text('data-test').endswith('Km'):
                diccionario['km'].append(kilom.get_text('data-test'))

        for desc in precios_descuento: # Metiendo los precios con descuento en el diccionario
            if "€" in desc.text:
                diccionario['precio_rebajado'].append(desc.text)

        coches_precios = [i.text for i in spans] # Creando list comprehension para nombres y precios originales de los coches
        counter = 0
        for i in coches_precios: # Añade un Null a la lista siempre que no haya un € donde debería haberlo
            counter += 1
            if counter %2 == 0 and "€" not in i:
                index = counter -1
                coches_precios.insert(index, "Null")
            else:
                pass
        
        for elemento in coches_precios: # Itera por la lista de coches y precios originales corregida con valores nulos y añade los valores a sus claves correspondientes en el diccionario
            if '€' in elemento or elemento == 'Null':
                diccionario['precio_original'].append(elemento)
            else:
                diccionario['coche'].append(elemento)
        
        if '€' not in coches_precios[-1]: # Añade el Null a los precios originales para contemplar el caso de que el último elemento no tenga precio original
            diccionario['precio_original'].append('Null')
                  
        return diccionario, pd.DataFrame(diccionario)  


url = "https://www.ocasionplus.com/coches-ocasion?v2&orderBy=morePopular"
coches = {'coche':[], 'precio_rebajado':[], 'precio_original':[], 'km':[]} # Crea diccionario para almacenar todos los datos del DataFrame
diccio, df_coches = scrape_coches_ocasion(url, coches)


In [110]:
df_coches

,coche,precio_rebajado,precio_original,km
0,BMW X2,24.818€,27.300€,53.681 Km
1,Volkswagen Multivan,26.900€,Null,144.649 Km
2,Hyundai Tucson,15.990€,17.590€,115.767 Km
3,Ford Puma,21.727€,23.900€,27.770 Km
4,KIA Picanto,10.681€,11.750€,103.355 Km
5,Land Rover Range Rover Evoque,17.900€,Null,150.251 Km
6,Nissan PULSAR,10.200€,Null,143.144 Km
7,Honda HR-V,15.636€,17.200€,87.125 Km
8,Opel Corsa,8.390€,Null,113.239 Km
9,Citroën C4 Cactus,9.909€,10.900€,90.716 Km


In [111]:
llamada = requests.get(url)
soup = BeautifulSoup(llamada.text, 'html.parser')
coches_precios = soup.find_all('span', {"class":"cardVehicle_spot__e6YZx"})

for coche in coches_precios:
    print(coche.text)

BMW X2
27.300€
Volkswagen Multivan
Hyundai Tucson
17.590€
Ford Puma
23.900€
KIA Picanto
11.750€
Land Rover Range Rover Evoque
Nissan PULSAR
Honda HR-V
17.200€
Opel Corsa
Citroën C4 Cactus
10.900€
Mercedes GLA
27.650€
KIA Ceed
12.800€
KIA Sportage
23.650€
Citroën C-Elysée
9.790€
Ford Focus
16.900€
Volkswagen Polo
11.990€
Peugeot 3008
20.450€
Mercedes Clase GLE
43.950€
Audi A1
27.000€
Opel Astra
